In [1]:
#預處理
import bs4 #網頁爬蟲常用套件，透過標籤取html檔案的資料
import requests #透過網址取得線上檔案，此處利用之取得網頁的html原始碼
import datetime as date #內建日期與時間的類別，格式化時間用
import json #自json讀取為dict，或將dict儲存為json

In [2]:
#時間格式化的函數，回傳格式化後的時間字串
def calTime(timeformat: str, cool3c_link="")-> str:
    """
    timeformat: 傳入來自technews.com的時間原始資料，若資料來自cool3c.com，則自串留空
    cool3c_link: 由於cool3c.com的文章發布確切時間得進入文章本身的網頁才可取得，而無法直接於目錄索引頁找到，故透過此參數傳入該文章的網址，若文章來自technews.com則不會用到此參數（預設為空字串）
    """
    currentTime = date.datetime.now()
    if cool3c_link != "": #透過連結處理cool3c.com之文章發布時間
        req = requests.get(cool3c_link) #取得文章網頁之html原始碼
        soup = bs4.BeautifulSoup(req.text,"html.parser") #將原始碼讀入為BeautifulSoup物件，方便取資料
        #取得的網頁原始碼（節錄）
        """
        上略
        <span class="icon">
            <i class="fas fa-bars"></i>
        </span>
        <span>硬科技：HotChips 32的新牙膏 IBM Power10與z15篇</span>
        <span class="prefix">by</span>
        <span class="icon">
            <i class="far fa-clock"></i>
        </span>
        <span class="">2020.09.23 12:01PM</span>
        <span class="icon">
            <i class="fab fa-line"></i>
        </span>
        下略
        """
        d = soup.select("span")[4]#由節錄可知時間資訊位於網頁的span標籤中的[4]的位置（第五項），故取之
        timeformat = d.text #讀取時間標籤內呈現的文字
        #原始字串：2020.09.23 12:01PM
        formatted = timeformat.replace(" ", ".").split(".") #格式化時間資訊字串，方便統一轉為datetime.datetime物件
        #處理為：["2020","09","23","12:01PM"]
        articleTime = date.datetime(int(formatted[0]), int(formatted[1]), int(formatted[2])) #轉換成datetime.datetime物件，僅保留年月日資訊
        return str(articleTime).split()[0] #回傳格式後的字串
    else: #透過目錄頁即可取得的時間資料處理來自technews.com的文章之發布時間
        #原始字串：2020 年 08 月 25 日 8:30
        formatted = timeformat.replace(" ", "").replace("年", " ").replace("月", " ").replace("日", " ").split()#格式化時間資訊字串，方便統一轉為datetime.datetime物件
        #處理為：["2020","08","25","8:30"]
        articleTime = date.datetime(int(formatted[0]), int(formatted[1]), int(formatted[2]))#轉換成datetime.datetime物件，僅保留年月日資訊
        return str(articleTime).split()[0] #回傳格式後的字串

以dict格式化要儲存的資料形式

In [ ]:
articles = {
        "update": str(date.datetime.now()), #上次更新時間
        "untagged_count": 0, #未有標籤的文章數目
        "count" : 0, #總文章數目
        "cool3c":[], #來自cool3c.com的文章資訊儲存處
        "technews": [] #來自technews.com的文章資訊儲存處
}
content_count = 0 #暫存文章累積數量
content = [] #各文章的內容摘要，因為資料是來自目次頁，故以list儲存
title = [] #各文章的標題
time = [] #各文章的發布時間
tags = [] #各文章的主題標籤

In [ ]:
print("Loading...", end="",flush=True)#表示要開始取資料了

取technews.com的資料，一頁最多15項

In [ ]:
req = requests.get("https://technews.tw/author/handymantw/")#先取第一頁，因為其網址最不同
soup = bs4.BeautifulSoup(req.text, "html.parser")#將原始碼讀入為BeautifulSoup物件，方便取資料
title = soup.select("h1.entry-title a")#取各文章標題
content = soup.select("div.moreinf")#取個文章內容摘錄
time = soup.select("span.body")[1::4]#取個文章發布時間資訊（在span.body裡面的[1]項，而後每四項出現一次）
tags = soup.select("span.body")[2::4]#取個文章的主題標籤（在span.body裡面的[2]項，而後每四項出現一次）
for i in range(len(title)): #逐篇儲存資訊
    print(".",end = "", flush=True)#視覺效果
    articles["technews"].append({#存入資料
        "title": title[i].text,#存文章標題
        "link": title[i]["href"],#存文章連結
        "content" : content[i].text[:-10].replace("&rdquo;", " ").replace("&ldquo;"," ")+"...", #存內容摘錄
        "time" : calTime(time[i].text), #存時間資訊，透過先前的函數格式化
        "tags" : list(set(tags[i].text.replace("\t","").replace("\n","").replace("\r","").replace(" ","").split(",")))# 存主題標籤
    })
    content_count+=1 #文章數目增加

In [ ]:
for j in range(2,100): #取目錄中其他頁的資訊
    req = requests.get("https://technews.tw/author/handymantw/page/"+str(j)+"/")
    soup = bs4.BeautifulSoup(req.text,"html.parser")
    title = soup.select("h1.entry-title a")
    content = soup.select("div.moreinf")
    time = soup.select("span.body")[1::4]
    tags = soup.select("span.body")[2::4]
    for i in range(len(title)):
        print(".",end = "", flush=True)
        articles["technews"].append({
            "title": title[i].text,
            "link": title[i]["href"],
            "content" : content[i].text[:-10].replace("&rdquo;", " ").replace("&ldquo;"," ")+"...",
            "time" : calTime(time[i].text),
            "tags" : list(set(tags[i].text.replace("\t","").replace("\n","").replace("\r","").replace(" ","").split(",")))
        })
        content_count+=1
    if(len(soup.select("h1.entry-title a")) < 15):#由於每頁最多15項，假若某一頁的項目不足15，則判斷已到達最後一頁
        break

取cool3.com的資料，一頁最多十項

In [ ]:
#有些文章沒有主題標籤，故先窮舉出來，方便後續例外處理
untagged = [
    "https://www.cool3c.com/article/159097",
    "https://www.cool3c.com/article/156566",
    "https://www.cool3c.com/article/156569",
    "https://www.cool3c.com/article/156568",
    "https://www.cool3c.com/article/156564",
    "https://www.cool3c.com/article/156563"
]#因為採窮舉，後續若有出現新的未標籤文章，則需更動程式碼，缺乏彈性
articles["untagged_count"] = len(untagged)#記錄有多少未有主題標籤的文章

In [ ]:
req = requests.get("https://www.cool3c.com/author/tiramisu")#先取第一頁，因為其網址最不同
soup = bs4.BeautifulSoup(req.text, "html.parser")#將原始碼讀入為BeautifulSoup物件，方便取資料
title = soup.select("div.title a")#取各文章標題
content = soup.select("div.content a")#取個文章內容摘錄
tags = soup.select("ul.list-inline")[3:]#取個文章的主題標籤（從ul.list-inline的[3]項開始）
for i in range(len(title)):#逐篇儲存資訊
    print(".",end = "", flush=True)#視覺效果
    if title[i]["href"] in untagged:#無標籤的例外處理1
        tags.insert(i, tags[0])#插入一項其他文章的標籤，避免後續文章取錯
    articles["cool3c"].append({#存入資料
        "title": title[i].text,#存文章標題
        "link": title[i]["href"],#存文章連結
        "content" : content[i].text.replace("&rdquo;", " ").replace("&ldquo;"," ")+"...", #存內容摘錄
        "time" : calTime("",cool3c_link=title[i]["href"])#存時間資訊，透過先前的函數格式化
    })
    if title[i]["href"] in untagged:#無標籤的例外處理1
        articles["cool3c"][-1]["tags"] = []#若文章本身無標籤，則將 articles["cool3c"][-1]["tags"]（最近插入的文章的「標籤」資料）設為空清單
    else:
        articles["cool3c"][-1]["tags"] = list(set(tags[i].text.replace("\n\n", "").split("\n")))#若文章有標籤，則格式化插入之
    content_count+=1#文章數目增加

In [ ]:
#取其他頁的資訊
for j in range(1,100):
    req = requests.get("https://www.cool3c.com/author/tiramisu/p"+str(j))
    soup = bs4.BeautifulSoup(req.text,"html.parser")
    title = soup.select("div.title a")
    content = soup.select("div.content a")
    temp = [elm.text for elm in soup.select("ul.list-inline")]
    tags = soup.select("ul.list-inline")[temp.index("\n\n\n\n")+1:]
    for i in range(len(title)):
        if title[i]["href"] in untagged:
            tags.insert(i, tags[0])
        if title[i].text == "訂閱硬科技電子報！":#裡面有一則廢文，跳過之
            continue
        print(".",end = "", flush=True)
        articles["cool3c"].append({
            "title": title[i].text, 
            "link": title[i]["href"], 
            "content" : content[i].text.replace("&rdquo;", " ").replace("&ldquo;"," ")+"...", 
            "time" : calTime("",cool3c_link=title[i]["href"]),
        })
        if title[i]["href"] in untagged:
            articles["cool3c"][-1]["tags"] = []
        else:
            articles["cool3c"][-1]["tags"] = list(set(tags[i].text.replace("\n\n", "").split("\n")))
        content_count+=1
    if(len(soup.select("div.title a")) < 10):#由於每頁最多10項，假若某一頁的項目不足10，則判斷已到達最後一頁
        break

輸出檔案

In [ ]:
articles["count"] = content_count#儲存文章總數目
with open("waterball.json", "w") as f:#開啟檔案寫成json
    json.dump(articles,f,indent=2)
    print("\nsaved.")